In [4]:
import redis
import pickle
import time
import pandas as pd
import os
from datetime import datetime

# ---------------------- 配置参数 ----------------------
REDIS_CONFIG_PATH = "redis.conf"  # Redis配置文件路径
TASK_QUEUE = "function_calls"
DAILY_K_PATH = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet"  # 日K线数据路径

# 队列控制参数 - 调整为适应20个worker的频次
MAX_QUEUE_SIZE = 10000      # 队列最大缓存任务数（增大以适应更多worker）
BATCH_SIZE = 100            # 每批发送任务数（增大）
SEND_INTERVAL = 1           # 批次发送间隔（缩短）
CHECK_INTERVAL_WHEN_FULL = 5 # 队列满时的检查间隔（缩短）

# ---------------------- 工具函数 ----------------------
def load_redis_config(config_path):
    """从redis.conf文件加载Redis配置（host=xxx格式）"""
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Redis配置文件不存在：{config_path}")
    
    # 默认配置
    host = "localhost"
    port = 6379
    password = ""
    
    # 读取并解析配置文件
    with open(config_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            # 跳过空行和注释
            if not line or line.startswith('#'):
                continue
            # 解析host
            if line.startswith('host='):
                host = line.split('=', 1)[1].strip()  # 只按第一个=分割
            # 解析port
            elif line.startswith('port='):
                try:
                    port = int(line.split('=', 1)[1].strip())
                except ValueError:
                    print(f"警告：port配置格式错误，使用默认值{port}")
            # 解析password
            elif line.startswith('password='):
                password = line.split('=', 1)[1].strip()
    
    return {
        "host": host,
        "port": port,
        "password": password,
        "decode_responses": False,
        "socket_timeout": 30,
        "socket_keepalive": True
    }

def load_valid_daily_data(parquet_path):
    """从日K线数据中加载有效的交易日和股票代码（paused==0）"""
    if not os.path.exists(parquet_path):
        raise FileNotFoundError(f"日K线文件不存在：{parquet_path}")
    
    print(f"开始加载日K线数据：{parquet_path}")
    # 只加载需要的列，提高效率
    df = pd.read_parquet(
        parquet_path,
        columns=['date', 'stock_code', 'paused']
    )
    
    # 筛选未停牌的数据（paused==0）
    valid_df = df[df['paused'] == 0].copy()

    valid_df = valid_df[valid_df['date'] >= '2025-01-01']

    # 格式化日期为YYYYMMDD
    valid_df['date'] = valid_df['date'].dt.strftime('%Y%m%d')
    
    # 筛选出2025年10月1日之后的数据
    valid_df = valid_df[valid_df['date'] >= '20251001']

    # 去重并按日期和股票代码分组
    valid_tasks = valid_df.groupby(['date', 'stock_code']).size().reset_index()
    valid_tasks = valid_tasks[['date', 'stock_code']]
    
    print(f"日K线数据加载完成，有效任务数：{len(valid_tasks)}")
    return valid_tasks

# ---------------------- 任务发送类 ----------------------
class IntelligentTaskSender:
    def __init__(self, redis_config):
        self.redis = redis.Redis(** redis_config)
        self._test_connection()
        # 使用Redis哈希存储元信息
        self.redis_metadata_key = "task_metadata"

    def _test_connection(self):
        """测试Redis连接"""
        try:
            self.redis.ping()
            print("✅ Redis连接成功")
        except Exception as e:
            print(f"❌ Redis连接失败：{e}")
            raise SystemExit(1)

    def _get_queue_length(self):
        """获取当前任务队列长度"""
        try:
            return self.redis.llen(TASK_QUEUE)
        except Exception as e:
            print(f"获取队列长度失败：{e}")
            return MAX_QUEUE_SIZE  # 失败时默认队列已满

    def send_tasks(self, valid_tasks):
        """根据有效任务列表发送任务，控制发送频率和队列长度"""
        total_tasks = len(valid_tasks)
        sent_count = 0
        print(f"开始发送任务，总有效任务数：{total_tasks}")

        while sent_count < total_tasks:
            # 检查队列容量
            current_len = self._get_queue_length()
            if current_len >= MAX_QUEUE_SIZE:
                print(f"⚠️ 队列已满（当前{current_len}/{MAX_QUEUE_SIZE}），暂停{CHECK_INTERVAL_WHEN_FULL}秒...")
                time.sleep(CHECK_INTERVAL_WHEN_FULL)
                continue

            # 计算本次可发送数量
            remaining = total_tasks - sent_count
            available = MAX_QUEUE_SIZE - current_len
            batch_count = min(remaining, available, BATCH_SIZE)

            # 提取批次任务
            batch = valid_tasks.iloc[sent_count:sent_count + batch_count]
            task_bytes_list = []
            
            for idx in range(len(batch)):
                task_id = f"task_{sent_count + idx}"
                trade_date = batch.iloc[idx]['date']
                stock_code = batch.iloc[idx]['stock_code']
                
                # 构建任务
                task = {
                    "func_name": "fetch_minute_stock_data",
                    "args": (trade_date, [stock_code]),
                    "kwargs": {},
                    "task_id": task_id
                }
                task_bytes_list.append(pickle.dumps(task))
                
                # 记录元数据到Redis
                metadata_value = pickle.dumps((trade_date, stock_code))
                self.redis.hset(self.redis_metadata_key, task_id, metadata_value)

            # 批量发送
            try:
                self.redis.rpush(TASK_QUEUE, *task_bytes_list)
                sent_count += batch_count
                
                # 打印进度
                progress = (sent_count / total_tasks) * 100
                print(f"📤 已发送 {sent_count}/{total_tasks} ({progress:.2f}%)，队列当前长度：{self._get_queue_length()}")
                
                # 批次发送间隔（不是最后一批才休眠）
                if sent_count < total_tasks:
                    time.sleep(SEND_INTERVAL)

            except Exception as e:
                print(f"❌ 批量发送失败：{e}，将重试当前批次")
                time.sleep(5)

        print("✅ 所有任务发送完成")

# ---------------------- 主函数 ----------------------
if __name__ == "__main__":
    try:
        # 1. 加载Redis配置
        redis_config = load_redis_config(REDIS_CONFIG_PATH)
        print(f"已加载Redis配置：host={redis_config['host']}, port={redis_config['port']}")
        
        # 2. 从日K线数据加载有效任务
        valid_tasks = load_valid_daily_data(DAILY_K_PATH)
        
        # 3. 发送任务
        sender = IntelligentTaskSender(redis_config)
        sender.send_tasks(valid_tasks)
        
    except Exception as e:
        print(f"程序执行失败：{e}")
        raise SystemExit(1)
    

已加载Redis配置：host=220.203.1.124, port=6379
开始加载日K线数据：D:\workspace\xiaoyao\data\stock_daily_price.parquet
日K线数据加载完成，有效任务数：10288
✅ Redis连接成功
开始发送任务，总有效任务数：10288
📤 已发送 100/10288 (0.97%)，队列当前长度：80
📤 已发送 200/10288 (1.94%)，队列当前长度：80
📤 已发送 300/10288 (2.92%)，队列当前长度：80
📤 已发送 400/10288 (3.89%)，队列当前长度：80
📤 已发送 500/10288 (4.86%)，队列当前长度：82
📤 已发送 600/10288 (5.83%)，队列当前长度：80
📤 已发送 700/10288 (6.80%)，队列当前长度：80
📤 已发送 800/10288 (7.78%)，队列当前长度：81
📤 已发送 900/10288 (8.75%)，队列当前长度：82
📤 已发送 1000/10288 (9.72%)，队列当前长度：86
📤 已发送 1100/10288 (10.69%)，队列当前长度：80
📤 已发送 1200/10288 (11.66%)，队列当前长度：80
📤 已发送 1300/10288 (12.64%)，队列当前长度：82
📤 已发送 1400/10288 (13.61%)，队列当前长度：81
📤 已发送 1500/10288 (14.58%)，队列当前长度：84
📤 已发送 1600/10288 (15.55%)，队列当前长度：87
📤 已发送 1700/10288 (16.52%)，队列当前长度：81
📤 已发送 1800/10288 (17.50%)，队列当前长度：81
📤 已发送 1900/10288 (18.47%)，队列当前长度：82
📤 已发送 2000/10288 (19.44%)，队列当前长度：84
📤 已发送 2100/10288 (20.41%)，队列当前长度：80
📤 已发送 2200/10288 (21.38%)，队列当前长度：81
📤 已发送 2300/10288 (22.36%)，队列当前长度：80
📤 已发送 2400/10288 (23.33%)，队列当前长度：8